앙상블 : 정형데이터의 가장 뛰어난 성과를 내는 알고리즘!
대부분 결정트리 기반으로 만들어짐
voting, bagging, boosting, stacking

랜덤포레스트 : 결정트리의 모음, 앙상블 학습시 제일 먼저 시도하기!
- 부트스트랩 샘플 : 중복한 샘플도 테스트

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('http://bit.ly/wine_csv_data')
data=wine[['alcohol', 'sugar', 'pH']]
target = wine['class']
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_jobs=-1, random_state=42)

In [3]:
scores = cross_validate(rf, train_input, train_target,
                       return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


랜덤 포레스트는 결정 트리의 앙상블이기 때문에 DecisionTreeClassifier가 제공하는 중요한 매개
변수를 모두 제공합니다. criterion, max_depth, max_features, min_samples_split, min_impurity_decrease, 
min_samples_leaf 등입니다. 또한 결정 트리의 큰 장점 중 하나인 특성 중요도를 계산합니다. 

In [4]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


자체적으로 모델을 평가하는 점수를 얻을 수 있습니다. 랜덤 포레스트는 훈련 세트에서 중복을 허용하여 부트스트랩 샘플을 만들어
결정 트리를 훈련한다고 했습니다. 이때 부트스트랩 샘플에 포함되지 않고 남는 샘플 = OOB(out of bag) 검증 세트
이 점수를 얻으려면 RandomForestClassifier 클래스의 oob_score 매개변수를 True로 지정
해야 합니다(이 매개변수의 기본값은 False입니다). 이렇게 하면 랜덤 포레스트는 각 결정 트리의
OOB 점수를 평균하여 출력합니다. oob_score=Tme로 지정하고 모델을 훈련하여 OOB 점수를
출력해 보겠습니다.


In [5]:
rf=RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


In [6]:
#엑스트라트리 : 부트스트랩과 다르게 중복된 샘플 없음. 한번씩만 사용 -> 무작위로 분할! 진정한 랜덤, 진정한 과대적합 방지
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#연산 속도가 빠름!



0.9974503966084433 0.8887848893166506


In [7]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


In [8]:
#그레디언트 부스팅: 깊이는 얕고, 경사하강법 적용 -> 결정트리를 계속 추가함
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [9]:
gb=GradientBoostingClassifier(n_estimators=500, learning_rate=0.2,
                              random_state=42)
scores=cross_validate(gb, train_input, train_target,
                      return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#결정 트리 개수를 500개, 학습률 learning rate의 기본값은 0.1 인데 0.2로 상향함

0.9464595437171814 0.8780082549788999


In [10]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15887763 0.6799705  0.16115187]


트리 훈련에 사용할 훈련 세트의 비율을 정하는 것은 subsample이고 
이 매개변수의 기본값은 1.0으로 전체 훈련 세트를 사용합니다. 하지만 subsample이 1보다
작으면 훈련 세트의 일부를 사용합니다. 이는 마치 경사 하강법 단계마다 일부 샘플을 랜덤하게 선택
하여 진행하는 확률적 경사 하강법 이나 미니 배치 경사 하강법과 비슷합니다. -> 이게 무슨 소리인가??

In [11]:
#그레디언트 부스팅에서 속도와 성능을 개선한게 히스토그램 기반 그레디언트 부스팅
#정형 데이터의 앙상블 중 가장 인기있음.
#특성을 255+1로 나눔으로써 노드 분할 시 최적의 노드를 빨리 찾을 수 있음->평행하게 주욱~ 늘려서 최적점 찾기
#XGBoost, LightGBM 있음!
from sklearn.ensemble import HistGradientBoostingClassifier
hgb=HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target,
                        return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#여기서는 n_jobs 지정을 안해도 되는군...

0.9321723946453317 0.8801241948619236


In [12]:
#특성중요도를 permutationJmportance () 함수로 계산해야하는 번거로움이 있음!
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result=permutation_importance(hgb, train_input, train_target,
                              n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)
#훈련레벨에서 중요도 체크

[0.08876275 0.23438522 0.08027708]


In [13]:
result = permutation_importance(hgb, test_input, test_target,
                                n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)
#테스트레벨에서 중요도 체크

[0.05969231 0.20238462 0.049     ]


In [14]:
#최종 성능 확인하자!!
hgb.score(test_input, test_target)

0.8723076923076923

In [16]:
from xgboost import XGBClassifier
xgb=XGBClassifier(tree_method='hist', random_state=42)
scores=cross_validate(xgb, train_input, train_target,
                      return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9567059184812372 0.8783915747390243


In [18]:
from lightgbm import LGBMClassifier
lgb=LGBMClassifier(random_state=42)
scores=cross_validate(lgb, train_input, train_target,
                      return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

[LightGBM] [Info] Number of positive: 3151, number of negative: 1006
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000160 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 370
[LightGBM] [Info] Number of data points in the train set: 4157, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.757999 -> initscore=1.141738
[LightGBM] [Info] Start training from score 1.141738
[LightGBM] [Info] Number of positive: 3151, number of negative: 1007
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.027474 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 373
[LightGBM] [Info] Number of data points in the train set: 4158, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.757816 -> initscore=1.140744
[LightGBM] [Info] Number of positive: 3151, number of negative: 1006
[LightGBM]